## twitter sentiment analysis
Create a labeled tweet dataset from api search results about any topic. Save clean tweets as .csv

In [5]:
import tweepy
from twitter_scraper import get_tweets
import json
from textblob import TextBlob
import pandas as pd

#### Authentication
Twitter requires user authentication to access its API. These keys can be found in the details of each app. https://developer.twitter.com/en/apps

In [2]:
#Authenticate
"""
#create credentials file

credentials={}
credentials['CONSUMER_KEY'] = '...'
credentials['CONSUMER_SECRET'] = '...'
credentials['ACCESS_TOKEN'] = '...'
credentials['ACCESS_SECRET'] = '...'

with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)
"""

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)
    
#Authorization for tweepy
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

api = tweepy.API(auth)
my_info = api.me()

In [3]:
#search
query = input('Enter search query: ')
search_results = api.search(query)
text_list = []
polarities = []
subjectivities = []

for tweet in range(len(search_results)):
    text_list.append(search_results[tweet].text)
    analysis = TextBlob(search_results[tweet].text)
    polarities.append(analysis.sentiment.polarity)
    subjectivities.append(analysis.sentiment.subjectivity)

In [4]:
#convert lists, reshape, and save results to CSV
text_list = pd.Series(text_list)
polarities = pd.Series(polarities)
subjectivities = pd.Series(subjectivities)

text_df = pd.DataFrame(data=[text_list, polarities, subjectivities]).T
text_df.columns = ['text', 'polarity', 'subjectivity']

text_df.to_csv(str(query) +'_tweets.csv')

## Sentiment by user without authentication

In [7]:
def user_scrape(user, n_pages):
    """#scrape the last n pages ( < 25) of tweets from specified Twitter user, analyse sentiment and save to csv for further processing"""
    user_tweets = []
    
    for tweet in get_tweets(user, pages=n_pages):
        analysis = TextBlob(tweet['text'])
        user_tweets.append({'text':tweet['text'], 'polarity':"{0:.2f}".format(round(analysis.sentiment.polarity,2)), 'subjectivity':"{0:.2f}".format(round(analysis.sentiment.subjectivity,2))})

    tweet_df = pd.DataFrame(user_tweets)
    tweet_df = tweet_df[['text', 'polarity', 'subjectivity']]
    tweet_df.to_csv(str(user) +'_tweets.csv')

In [8]:
user_scrape('nytimes', 10)